# Librerías

In [1]:
import re

from functools import reduce

import numpy as np
import pandas as pd

import urllib
import json
from bs4 import BeautifulSoup

from sqlalchemy import create_engine
from sqlalchemy.types import CHAR,VARCHAR,INTEGER,DATE,DATETIME,BOOLEAN,FLOAT,NUMERIC,TEXT,BIGINT

import json

# Extracción de datos de la web

Extracción de datos correspondientes a episodios de los Simpsons de las temporadas 1 a 30. 

Los datos fueron extraídos de la página web  **https://simpsons.fandom.com/wiki/List_of_Episodes**

In [2]:
episodes_url='https://simpsons.fandom.com/wiki/List_of_Episodes'

In [3]:
html=urllib.request.urlopen(episodes_url).read()

In [4]:
seasons=BeautifulSoup(html,'html.parser')('tbody')[:30] # datos (tabla de episodios) de las temporadas 1 to 30

In [5]:
def GetEpisodes(data:list=seasons)->pd.DataFrame:
    """Lee las tablas, extrae el url,titulo,fecha y codigo de produccion de cada episodio

    Args:
        data:  Lista con las tablas(temporadas). Defaults to seasons:list.

    Returns:
        pd.DataFrame: DatatFrame con los episodios
    """

    regex='.*?href="(.*?)"\s+title=.*?>(.*?)<.*?<center>(.*?)<.*?<center>(.*?)<'
    re_obj = re.compile(regex)
    temps=[]

    for num,tem in enumerate(seasons,start=1):
        aux=seasons[num-1]('tr')
        data=reduce(lambda x,y:x+y,map(lambda x:re_obj.findall(x.decode().replace('\n','')),aux))
        df=pd.DataFrame(data,columns=['url','title','date','code'])
        df.insert(0,'season',num)
        df['url']=df['url'].map(lambda x:'https://simpsons.fandom.com'+x)
        df['season_episodes']=df.index+1
        temps.append(df)

    return pd.concat(temps).query('code!="--"')

In [6]:
data=GetEpisodes()
data['title']=data['title'].str.replace('&amp;','&')
data.reset_index(drop=True,inplace=True)
data.shape

(661, 6)

In [7]:
data.head(3)

,season,url,title,date,code,season_episodes
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,"December 17, 1989",7G08,1
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,"January 14, 1990",7G02,2
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,"January 21, 1990",7G03,3


In [8]:
re_wd= re.compile('.*?Written By(.*?)Directed By(.*)')
def WrittenDirector(url:str)->tuple:
    """EXtrae los directores y escritores del episodio 

    Args:
        url (str): url con el link del episodio

    Returns:
        tuple: tupla (escritores,directores)
    """
    html=urllib.request.urlopen(url).read()
    data=BeautifulSoup(html,'html.parser')('aside')[0].text.replace('\n','')
    return re_wd.findall(data)[0]

In [9]:
data['writter_director']=pd.DataFrame(data['url'].apply(lambda x:WrittenDirector(x)))
data.head(3)

,season,url,title,date,code,season_episodes,writter_director
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,"December 17, 1989",7G08,1,"(Mimi Pond, David Silverman)"
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,"January 14, 1990",7G02,2,"(Jon Vitti, David Silverman)"
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,"January 21, 1990",7G03,3,"(Jay Kogen & Wallace Wolodarsky, Wes Archer)"


In [10]:
def Synopsis(url:str)->str:
    """Extrae un resumen del episodio

    Args:
        url (str): url con el link del episodio

    Returns:
        str: Resumen del episodio
    """
    html=urllib.request.urlopen(url).read()
    data=BeautifulSoup(html,'html.parser')

    try:
        sy=re.findall('<h2>.*?id="[SP][a-z]*".*?</h2><p>(.*?)</p>',data.decode().replace('\n','').replace('\t',''))[0]
    except:
        sy=''

    if sy=='':
        try:
            sy=re.findall('<h2>.*?id="[SP][a-z]*".*?</h2>.*?</figure><p>(.*?)</p>',data.decode().replace('\n','').replace('\t',''))[0]
        except:
            pass

    return re.sub('<.*?>','',sy)

In [11]:
data['synopsis']=data['url'].map(lambda x:Synopsis(x))
data.head(3)

,season,url,title,date,code,season_episodes,writter_director,synopsis
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,"December 17, 1989",7G08,1,"(Mimi Pond, David Silverman)",When Mr. Burns announces that none of the work...
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,"January 14, 1990",7G02,2,"(Jon Vitti, David Silverman)","To get back at class nerd/teacher's pet, Marti..."
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,"January 21, 1990",7G03,3,"(Jay Kogen & Wallace Wolodarsky, Wes Archer)",After being fired from his job at the nuclear ...


In [12]:
def Story(url:str)->str:
    """Extrae la historia completa del episodio

    Args:
        url (str): url del episodio

    Returns:
        str: Historia del episodio
    """

    html=urllib.request.urlopen(url).read()
    data=BeautifulSoup(html,'html.parser')
    aux=re.split('<h2>.*?id="[Full_StoryPlotsSynopsis]*".*?</h2>',data.decode())[-1].split('<h2>')[0]

    return re.sub('\s{2}',' ',re.sub('<.*?>','',aux.replace('\n','').replace('\t','')))

In [13]:
data['story']=data['url'].map(lambda x:Story(x))
data['num_episodes']=data.index+1
data.head(3)

,season,url,title,date,code,season_episodes,writter_director,synopsis,story,num_episodes
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,"December 17, 1989",7G08,1,"(Mimi Pond, David Silverman)",When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,"January 14, 1990",7G02,2,"(Jon Vitti, David Silverman)","To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,"January 21, 1990",7G03,3,"(Jay Kogen & Wallace Wolodarsky, Wes Archer)",After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3


In [14]:
ruta='/home/asm/kaggle/TheSimpsons/'
data.to_parquet(ruta+'data.parquet',engine='pyarrow')

# Transformación y limpieza

In [136]:
ruta='/home/asm/kaggle/TheSimpsons/'
df=pd.read_parquet(ruta+'data.parquet',engine='pyarrow')
df.head()

,season,url,title,date,code,season_episodes,writter_director,synopsis,story,num_episodes
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,"December 17, 1989",7G08,1,"[Mimi Pond, David Silverman]",When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,"January 14, 1990",7G02,2,"[Jon Vitti, David Silverman]","To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,"January 21, 1990",7G03,3,"[Jay Kogen & Wallace Wolodarsky, Wes Archer]",After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,"January 28, 1990",7G04,4,"[Al Jean & Mike Reiss, Gregg Vanzo & Kent Butt...",After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4
4,1,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,"February 4, 1990",7G05,5,"[John Swartzwelder, David Silverman]",After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The...,5


In [137]:
df.shape

(661, 10)

In [138]:
df['title']=df['title'].str.strip()
df['date']=pd.to_datetime(df.date.str.replace(',',''))
df.head()

,season,url,title,date,code,season_episodes,writter_director,synopsis,story,num_episodes
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,"[Mimi Pond, David Silverman]",When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"[Jon Vitti, David Silverman]","To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,"[Jay Kogen & Wallace Wolodarsky, Wes Archer]",After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,"[Al Jean & Mike Reiss, Gregg Vanzo & Kent Butt...",After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4
4,1,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,1990-02-04,7G05,5,"[John Swartzwelder, David Silverman]",After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The...,5


In [139]:
df['writters']=df['writter_director'].map(lambda x:x[0])
df['directors']=df['writter_director'].map(lambda x:x[1])

df['writters']=df['writters'].str.replace('&',',')  # remplazar & por ,
df['directors']=df['directors'].str.replace('&',',')

df['writters']=df['writters'].map(lambda x:re.sub('\(.*?\)',',',x)) # rempazar partentesis con coma
df['directors']=df['directors'].map(lambda x:re.sub('\(.*?\)',',',x))

df['writters']=df['writters'].map(lambda x:re.sub('\[.*?\]','',x)) # eliminar []
df['directors']=df['directors'].map(lambda x:re.sub('\[.*?\]','',x))


df['writters']=df['writters'].map(lambda x:re.sub('Credits','',x)) # eliminar la palabra Credits
df['directors']=df['directors'].map(lambda x:re.sub('Credits','',x))

df['writters']=df['writters'].map(lambda x:re.sub('\d','',x)) # eliminar numeros
df['directors']=df['directors'].map(lambda x:re.sub('\d','',x))

df['writters']=df['writters'].map(lambda x:re.sub('\S:','',x)) # eliminar palabras seguidas por :


df.head()

,season,url,title,date,code,season_episodes,writter_director,synopsis,story,num_episodes,writters,directors
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,"[Mimi Pond, David Silverman]",When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,Mimi Pond,David Silverman
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"[Jon Vitti, David Silverman]","To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,Jon Vitti,David Silverman
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,"[Jay Kogen & Wallace Wolodarsky, Wes Archer]",After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,"Jay Kogen , Wallace Wolodarsky",Wes Archer
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,"[Al Jean & Mike Reiss, Gregg Vanzo & Kent Butt...",After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,"Al Jean , Mike Reiss","Gregg Vanzo , Kent Butterworth"
4,1,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,1990-02-04,7G05,5,"[John Swartzwelder, David Silverman]",After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The...,5,John Swartzwelder,David Silverman


### Corregir el nombre de los directores

In [140]:
df['directors']=df['directors'].str.replace('Wesley','Wes')
df['directors']=df['directors'].str.replace('Steven','Steve')
df['directors']=df['directors'].str.replace('B.','Beheaded')
df['directors']=df['directors'].str.replace('Jeffrey','Jeff')
df['directors']=df['directors'].str.replace('and',',')
df['directors']=df['directors'].str.replace('Bedlam','')
df['directors']=df['directors'].str.replace('Bloodcurdling','')
df['directors']=df['directors'].str.replace('Manic','')
df['directors']=df['directors'].str.strip()

In [141]:
def nombres(name:str)->str:
    """Separa los nombre que estan juntos con una coma
    Ejemplo:
            Seth RogenEvan Goldberg -> Seth Rogen , Evan Goldberg

    Args:
        name (str): string con el numbre

    Returns:
        str: string con los nombres separados por una coma
    """
    aux=re.findall('[a-z][A-Z]',name)
    if len(aux)==0:
        return name
    else:
        for x in aux:
            name=re.sub(x,x[0]+' , '+x[-1],name)
        return name

df['directors']=df['directors'].map(nombres)

In [142]:
df['directors']=df['directors'].map(lambda x:re.sub(',$','',x)) # elimina , al final
df['directors']=df['directors'].map(lambda x:re.sub('^,','',x)) # elimina , al inicio
df['directors']=df['directors'].str.strip()
df['directors']=df['directors'].map(lambda x:re.sub('".*?"','',x)) # elimina texto entre comillas
df['directors']=df['directors'].map(lambda x:re.sub(',\s{0,1},',',',x)) # remplaza espacios en blanco entre comas con una coma
df['directors']=df['directors'].map(lambda x:re.sub('\s{2}',' ',x)) # elimina mas de un espacio en blanco
df.head()

,season,url,title,date,code,season_episodes,writter_director,synopsis,story,num_episodes,writters,directors
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,"[Mimi Pond, David Silverman]",When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,Mimi Pond,David Silverman
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"[Jon Vitti, David Silverman]","To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,Jon Vitti,David Silverman
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,"[Jay Kogen & Wallace Wolodarsky, Wes Archer]",After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,"Jay Kogen , Wallace Wolodarsky",Wes Archer
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,"[Al Jean & Mike Reiss, Gregg Vanzo & Kent Butt...",After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,"Al Jean , Mike Reiss","Gregg Vanzo , Kent Butterworth"
4,1,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,1990-02-04,7G05,5,"[John Swartzwelder, David Silverman]",After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The...,5,John Swartzwelder,David Silverman


### Corregir el nombre de los escritores

In [143]:
df['writters']=df['writters'].str.replace('Edward Danielchunhands','Daniel Chun')
df['writters']=df['writters'].str.replace('Atrocious','')
df['writters']=df['writters'].str.replace('Jittery','')
df['writters']=df['writters'].str.replace('Gasping','')
df['writters']=df['writters'].str.replace('Slithering','')
df['writters']=df['writters'].str.replace('Spooky','')
df['writters']=df['writters'].str.replace('Warped','')
df['writters']=df['writters'].str.replace('Scarifying','')
df['writters']=df['writters'].str.replace('Katastrophe','')
df['writters']=df['writters'].str.replace('Vicious','')
df['writters']=df['writters'].str.replace('Morbid','')
df['writters']=df['writters'].str.replace('Scary','')

df['writters']=df['writters'].str.strip()

In [144]:
df['writters']=df['writters'].str.replace('Al Jean  Mike Reiss','Al Jean , Mike Reiss')
df['writters']=df['writters'].str.replace('and',',')

In [145]:
df['writters']=df['writters'].map(lambda x:re.sub(',$','',x)) # elimina , al final
df['writters']=df['writters'].map(lambda x:re.sub('^,','',x)) # elimina , al inicio
df['writters']=df['writters'].map(lambda x:re.sub('".*?"','',x)) # elimina texto entre comillas
df['writters']=df['writters'].map(lambda x:re.sub(',\s{0,1},',',',x)) # remplaza espacios en blanco entre comas con una coma
df['writters']=df['writters'].map(lambda x:re.sub('\s{2}',' ',x)) # elimina mas de un espacio en blanco
df['writters']=df['writters'].str.strip()

df['writters']=df['writters'].map(nombres)

In [146]:
df['writters']=df['writters'].str.replace('Dan Mc , Grath','Dan McGrath')
df['writters']=df['writters'].str.replace('David S. Cohen','David Cohen')
df['writters']=df['writters'].str.replace('David X. Cohen','David Cohen')
df['writters']=df['writters'].str.replace('M,el','Mel')
df['writters']=df['writters'].str.replace('Rob La , Zebnik','Rob LaZebnik')
df['writters']=df['writters'].str.replace('Mir,a','Mira')
df['writters']=df['writters'].str.replace('Mc , Connachie','McConnachie')
df['writters']=df['writters'].str.replace(', under the pseudonym','')
df['writters']=df['writters'].str.replace('Story b','')
df['writters']=df['writters'].str.replace('Telepla','')
df['writters']=df['writters'].str.replace('Stor','')
df['writters']=df['writters'].str.replace('Valentina L. Garza','Valentina Garza')
df['writters']=df['writters'].str.replace('The Late','')
df['writters']=df['writters'].str.replace('The Estate of','')
df['writters']=df['writters'].str.replace('Disfigured','')
df['writters']=df['writters'].str.replace('Bilious','')
df['writters']=df['writters'].str.replace('Watch','')
df['writters']=df['writters'].str.replace('David²+S.²+Cohen²','David S. Cohen')
df['writters']=df['writters'].str.replace('Wally','Wallace')

In [147]:
df['writters']=[','.join(map(str.strip,x.split(','))) for x in df['writters']]
df['directors']=[','.join(map(str.strip,x.split(','))) for x in df['directors']]
df.drop(['writter_director'],axis=1,inplace=True)
df.head()

,season,url,title,date,code,season_episodes,synopsis,story,num_episodes,writters,directors
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,Mimi Pond,David Silverman
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,Jon Vitti,David Silverman
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,"Jay Kogen,Wallace Wolodarsky",Wes Archer
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,"Al Jean,Mike Reiss","Gregg Vanzo,Kent Butterworth"
4,1,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,1990-02-04,7G05,5,After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The...,5,John Swartzwelder,David Silverman


### Limpieza de las variables synopsis,story

In [148]:
df['story']=np.where(df['story'].map(len)==0,df.synopsis,df.story) # si la historia es vacia, usamos la sinopsisi
df['story']=df['story'].map(lambda x:re.sub('^.*Intro.*?\[\]','',x))  
df['story']=df['story'].str.replace('[]','') # eliminar [] 

In [149]:
df['story']=df['story'].str.replace('Opening Sequence','')
df['story']=df['story'].str.replace('Opening','')
df['story']=df['story'].str.replace('Opening . ','')
df['story']=df['story'].str.replace("The Episode's logo",'')
df['story']=df['story'].str.replace('The logo for this episode','')
df['story']=df['story'].str.replace('Opening segment ','')
df['story']=df['story'].str.replace('The logo for the episode.','')
df['story']=df['story'].str.replace("The episode's title card.",'')

In [150]:
df['story']=df['story'].str.strip()
df['story']=df['story'].map(lambda x:re.sub('^\.','',x)) # eliminar . al inicio
df['story']=df['story'].map(lambda x:re.sub('^\(.*?\)','',x)) # eliminar () al inicio
df['story']=df['story'].map(lambda x:re.sub('\(.*?\)$','',x)) # eliminar () al final
df['story']=df['story'].str.strip()

In [151]:
df['story']=df['story'].map(lambda x:x.split('Treehouse of Horror series')[0])
df['story']=df['story'].str.strip()

In [152]:
df.head()

,season,url,title,date,code,season_episodes,synopsis,story,num_episodes,writters,directors
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,Mimi Pond,David Silverman
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,Jon Vitti,David Silverman
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,"Jay Kogen,Wallace Wolodarsky",Wes Archer
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,"Al Jean,Mike Reiss","Gregg Vanzo,Kent Butterworth"
4,1,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,1990-02-04,7G05,5,After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The e...,5,John Swartzwelder,David Silverman


### Transdorma cada valor en las columnas writters,directors en un solo registro

For a DataFrame column with nested, list-like values, explode() will transform each list-like value to a separate row. The resulting Index will be duplicated corresponding to the index label from the original row:

In [153]:
df['writters']=df['writters'].str.split(',')
df['directors']=df['directors'].str.split(',')
df.head()

,season,url,title,date,code,season_episodes,synopsis,story,num_episodes,writters,directors
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,[Mimi Pond],[David Silverman]
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,[Jon Vitti],[David Silverman]
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,"[Jay Kogen, Wallace Wolodarsky]",[Wes Archer]
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,"[Al Jean, Mike Reiss]","[Gregg Vanzo, Kent Butterworth]"
4,1,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,1990-02-04,7G05,5,After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The e...,5,[John Swartzwelder],[David Silverman]


In [154]:
aux_w=df['writters'].explode()
aux_d=df['directors'].explode()

In [155]:
df=df.merge(aux_w,left_index=True,right_index=True).merge(aux_d,left_index=True,right_index=True)
df.head()

,season,url,title,date,code,season_episodes,synopsis,story,num_episodes,writters_x,directors_x,writters_y,directors_y
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,[Mimi Pond],[David Silverman],Mimi Pond,David Silverman
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,[Jon Vitti],[David Silverman],Jon Vitti,David Silverman
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,"[Jay Kogen, Wallace Wolodarsky]",[Wes Archer],Jay Kogen,Wes Archer
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,"[Jay Kogen, Wallace Wolodarsky]",[Wes Archer],Wallace Wolodarsky,Wes Archer
3,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,"[Al Jean, Mike Reiss]","[Gregg Vanzo, Kent Butterworth]",Al Jean,Gregg Vanzo


In [156]:
df.shape

(881, 13)

In [157]:
df.drop(['writters_x','directors_x'],axis=1,inplace=True)
df.rename(columns={'writters_y':'writter','directors_y':'director'},inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,season,url,title,date,code,season_episodes,synopsis,story,num_episodes,writter,director
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,Mimi Pond,David Silverman
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,Jon Vitti,David Silverman
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,Jay Kogen,Wes Archer
3,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,Wallace Wolodarsky,Wes Archer
4,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,Al Jean,Gregg Vanzo


In [158]:
df.shape

(881, 11)

In [159]:
df.to_parquet(ruta+'data_limpio.parquet',engine='pyarrow')

# Carga

In [2]:
ruta='/home/asm/kaggle/TheSimpsons/'
df=pd.read_parquet(ruta+'data_limpio.parquet',engine='pyarrow')
df.head()

,season,url,title,date,code,season_episodes,synopsis,story,num_episodes,writter,director
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,Mimi Pond,David Silverman
1,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,Jon Vitti,David Silverman
2,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,Jay Kogen,Wes Archer
3,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,Wallace Wolodarsky,Wes Archer
4,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,Al Jean,Gregg Vanzo


### Conexión a la Base de Datos

In [3]:
creds = json.load(open('../../creds_db.json','r'))
cnx = create_engine(f"mysql+pymysql://{creds['u']}:{creds['p']}@{creds['h']}/{creds['d']}").connect()

### Entidades
- Temporada (season)
- Escritor (writter)
- Director (director)
- Episodio (episode)

#### Entidad Temporada

In [4]:
temp=df.season.drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'id'})
temp['id']+=1
temp.head(3)

,id,season
0,1,1
1,2,2
2,3,3


In [5]:
dtypes = [INTEGER,INTEGER]
temp.to_sql(name='tbl_season',
               index=False,
               con=cnx,
               if_exists='replace',
               dtype=dict(zip(temp.columns,dtypes)))


30

#### Entidad Escritor

In [6]:
esc=df.writter.drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'id'})
esc['id']+=1
esc.head(3)

,id,writter
0,1,Mimi Pond
1,2,Jon Vitti
2,3,Jay Kogen


In [7]:
esc.writter.map(len).max()

19

In [8]:
dtypes = [INTEGER,VARCHAR(20)]
esc.to_sql(name='tbl_writter',
               index=False,
               con=cnx,
               if_exists='replace',
               dtype=dict(zip(esc.columns,dtypes)))

148

#### Entidad Director

In [9]:
direc=df.director.drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'id'})
direc['id']+=1
direc.head(3)

,id,director
0,1,David Silverman
1,2,Wes Archer
2,3,Gregg Vanzo


In [10]:
direc.director.map(len).max()

22

In [11]:
dtypes = [INTEGER,VARCHAR(22)]
direc.to_sql(name='tbl_director',
               index=False,
               con=cnx,
               if_exists='replace',
               dtype=dict(zip(direc.columns,dtypes)))

41

#### Entidad Episodio

In [12]:
ep=df.copy()
ep.insert(0,'id',ep.code.map({v:k for k,v in enumerate(ep.code.unique(),start=1)}))
ep.head()

,id,season,url,title,date,code,season_episodes,synopsis,story,num_episodes,writter,director
0,1,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,1,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1,Mimi Pond,David Silverman
1,2,1,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,2,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2,Jon Vitti,David Silverman
2,3,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,Jay Kogen,Wes Archer
3,3,1,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,3,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3,Wallace Wolodarsky,Wes Archer
4,4,1,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,4,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4,Al Jean,Gregg Vanzo


#### Tabla intermedia Episodio-Temporada

In [13]:
temp_ep=temp.merge(ep[['id','season','season_episodes']],on='season',how='inner').rename(columns={'id_x':'id_season','id_y':'id_episode'})\
    .drop_duplicates().reset_index(drop=True)
temp_ep.insert(0,'id',temp_ep.index+1)
temp_ep.drop(['season'],axis=1,inplace=True)
temp_ep.sample(3)

,id,id_season,id_episode,season_episodes
523,524,24,524,16
390,391,18,391,13
29,30,2,30,17


In [14]:
dtypes = [INTEGER]*4
temp_ep.to_sql(name='tbl_season_ep',
               index=False,
               con=cnx,
               if_exists='replace',
               dtype=dict(zip(temp_ep.columns,dtypes)))

661

#### Tabla intermedia Episodio-Escritor

In [15]:
ep_esc=esc.merge(ep[['id','writter']],on='writter',how='inner').rename(columns={'id_x':'id_writter','id_y':'id_episode'}).reset_index(drop=True)
ep_esc.drop('writter',axis=1,inplace=True)
ep_esc.insert(0,'id',ep_esc.index+1)
ep_esc.sample(3)

,id,id_writter,id_episode
436,437,66,411
276,277,36,99
413,414,64,221


In [16]:
dtypes = [INTEGER,INTEGER,INTEGER]
ep_esc.to_sql(name='tbl_writter_ep',
               index=False,
               con=cnx,
               if_exists='replace',
               dtype=dict(zip(ep_esc.columns,dtypes)))

881

#### Tabla intermedia Episodio-Director

In [17]:
ep_dir=direc.merge(ep[['id','director']],on='director',how='inner').rename(columns={'id_x':'id_director','id_y':'id_episode'}).reset_index(drop=True)
ep_dir.drop('director',axis=1,inplace=True)
ep_dir.insert(0,'id',ep_dir.index+1)
ep_dir.sample(5)

,id,id_director,id_episode
125,126,6,11
352,353,13,321
212,213,8,484
7,8,1,16
379,380,13,537


In [18]:
dtypes = [INTEGER,INTEGER,INTEGER]
ep_dir.to_sql(name='tbl_director_ep',
               index=False,
               con=cnx,
               if_exists='replace',
               dtype=dict(zip(ep_dir.columns,dtypes)))

881

#### Carga tabla Episodio

In [19]:
ep.drop(['season','season_episodes','writter','director'],axis=1,inplace=True)
ep.drop_duplicates(inplace=True)
ep.head()

,id,url,title,date,code,synopsis,story,num_episodes
0,1,https://simpsons.fandom.com/wiki/Simpsons_Roas...,Simpsons Roasting on an Open Fire,1989-12-17,7G08,When Mr. Burns announces that none of the work...,Homer hastily drives the Family Sedan with Mar...,1
1,2,https://simpsons.fandom.com/wiki/Bart_the_Genius,Bart the Genius,1990-01-14,7G02,"To get back at class nerd/teacher's pet, Marti...",The Simpson family is playing Scrabble in the ...,2
2,3,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,Homer's Odyssey,1990-01-21,7G03,After being fired from his job at the nuclear ...,The episode begins in front of Springfield Ele...,3
4,4,https://simpsons.fandom.com/wiki/There%27s_No_...,There's No Disgrace Like Home,1990-01-28,7G04,After being embarrassed by the rest of his fam...,"Bart and Lisa are fighting, but it is not long...",4
8,5,https://simpsons.fandom.com/wiki/Bart_the_General,Bart the General,1990-02-04,7G05,After defending Lisa from school bully Nelson ...,Bart and Lisa fight over Lisa's cupcakes The e...,5


In [20]:
ep[['url','title','code','synopsis','story']].map(len).max()

url           120
title          83
code            7
synopsis      937
story       22495
dtype: int64

In [21]:
dtypes = [INTEGER,VARCHAR(120),VARCHAR(85),DATE,VARCHAR(7),TEXT,TEXT,INTEGER]
ep.to_sql(name='tbl_episode',
               index=False,
               con=cnx,
               if_exists='replace',
               dtype=dict(zip(ep.columns,dtypes)))

661